In [1]:
!git clone https://github.com/Bhattars1/Restaurant-Till-Assistant/

Cloning into 'Restaurant-Till-Assistant'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 21 (delta 3), reused 15 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (21/21), 26.68 KiB | 440.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.


In [2]:
import pandas as pd
path = "Restaurant-Till-Assistant/Burger King Menu.csv"
menu = pd.read_csv(path)
menu.head()

,SN,Items,Price,Categories
0,1,Whopper,8,burger
1,2,Bacon Double X L,8,burger
2,3,PB Whopper,8,burger
3,4,Chilli Jam Whopper,10,burger
4,5,Ham Burger,2,burger


In [3]:
categories = list(menu["Categories"].unique())
print(categories)

['burger', 'sides', 'drinks', 'dips', 'fries']


In [4]:
customer_dataset = pd.read_csv("Restaurant-Till-Assistant/customer_till_interactions.csv")
customer_dataset.head()

,Customer_Speaking,Till_Response
0,"I need Fries, 6 Nuggets, 3 Chilli Bites","Sure! You've ordered Fries, 6 Nuggets, 3 Chill..."
1,"I need Bacon Stacker, medium Mayo","Sure! You've ordered Bacon Stacker, Mayo."
2,I need Large Fries,Sure! You've ordered Large Fries.
3,"Please give me small Hot Chilli, 9 Chilli Bites","Sure! You've ordered Hot Chilli, 9 Chilli Bites."
4,"I'll have Diet Cocacola, Steakhouse Angus, med...","Sure! You've ordered Diet Cocacola, Steakhouse..."


In [5]:
# Creating a new column extracting the ordered items from the customer

In [6]:
# Function to extract ordered items from Till_Response
def extract_items_from_response(till_response):
    # Convert the response to lowercase
    response_lower = till_response.lower()

    # Prepare a list to collect ordered items
    ordered_items = []

    # Check for direct item matches in the menu
    for item in menu['Items'].str.lower():
        if item in response_lower:
            ordered_items.append(item)

    return ', '.join(ordered_items)

# Create a new column with extracted items from Till_Response
customer_dataset['Ordered_Items'] = customer_dataset['Till_Response'].apply(extract_items_from_response)

# Export the modified customer_dataset to a CSV file
customer_dataset.to_csv('customer_orders.csv', index=False)

customer_dataset.head(20)

,Customer_Speaking,Till_Response,Ordered_Items
0,"I need Fries, 6 Nuggets, 3 Chilli Bites","Sure! You've ordered Fries, 6 Nuggets, 3 Chill...","3 chilli bites, 6 nuggets, fries"
1,"I need Bacon Stacker, medium Mayo","Sure! You've ordered Bacon Stacker, Mayo.","bacon stacker, mayo"
2,I need Large Fries,Sure! You've ordered Large Fries.,"fries, large fries"
3,"Please give me small Hot Chilli, 9 Chilli Bites","Sure! You've ordered Hot Chilli, 9 Chilli Bites.","9 chilli bites, hot chilli"
4,"I'll have Diet Cocacola, Steakhouse Angus, med...","Sure! You've ordered Diet Cocacola, Steakhouse...","steakhouse angus, 9 chilli bites, diet cocacol..."
5,I need 20 Nuggets,Sure! You've ordered 20 Nuggets.,20 nuggets
6,"Can I get 6 Chilli Bites, large Cocacola Zero,...","Sure! You've ordered 6 Chilli Bites, Cocacola ...","6 chilli bites, cocacola zero, fries"
7,"I want medium Steakhouse Angus, medium 9 Dorit...","Sure! You've ordered Steakhouse Angus, 9 Dorit...","steakhouse angus, 4 nuggets, 9 doritos, tea"
8,I'll have medium Steakhouse Chicken,Sure! You've ordered Steakhouse Chicken.,"steakhouse chicken, tea"
9,"I'll have Bacon Stacker, 6 Chilli Bites","Sure! You've ordered Bacon Stacker, 6 Chilli B...","bacon stacker, 6 chilli bites"


# Now what I will be doing is fine tune a GPT2 model for generating till response if any customer gives any prompt

In [7]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments, pipeline
!pip install datasets
from datasets import Dataset

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.0 MB/s eta 0:00:00


In [8]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

CONVERSION_PROMPT = "TILL\n"
CONVERSION_TOKEN_1 = "Order"
# CONVERSION_TOKEN_2 = "Items"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [15]:
# This is our "training prompt" that we want GPT2 to recognize and learn
training_examples = f'{CONVERSION_PROMPT}Query: ' + customer_dataset['Customer_Speaking'] + '\n' + CONVERSION_TOKEN_1 + ': ' + customer_dataset['Till_Response'].astype(str)

print(training_examples[0])

TILL
Query: I need Fries, 6 Nuggets, 3 Chilli Bites
Order: Sure! You've ordered Fries, 6 Nuggets, 3 Chilli Bites.


In [16]:
task_df = pd.DataFrame({"text": training_examples})
task_df.head()

,text
0,"TILL\nQuery: I need Fries, 6 Nuggets, 3 Chilli..."
1,"TILL\nQuery: I need Bacon Stacker, medium Mayo..."
2,TILL\nQuery: I need Large Fries\nOrder: Sure! ...
3,"TILL\nQuery: Please give me small Hot Chilli, ..."
4,"TILL\nQuery: I'll have Diet Cocacola, Steakhou..."


In [17]:
len(training_examples)

1000

In [18]:
customer_data = Dataset.from_pandas(task_df)  # turn a pandas DataFrame into a Dataset

def preprocess(examples):  # tokenize our text but don't pad because our collator will pad for us dynamically
    return tokenizer(examples['text'], truncation=True)

customer_data = customer_data.map(preprocess, batched=True)

customer_data = customer_data.train_test_split(train_size=.85)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [19]:
data_collator= DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
BKUK_TILL = GPT2LMHeadModel.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [20]:
training_args = TrainingArguments(
    output_dir="./baseline_till_assistant_model_v1",
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=2, # batch size for training
    per_device_eval_batch_size=20,  # batch size for evaluation
    load_best_model_at_end=True,
    logging_steps=5,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=BKUK_TILL,
    args=training_args,
    train_dataset=customer_data["train"],
    eval_dataset=customer_data["test"],
    data_collator=data_collator,
)

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(

***** Running Evaluation *****
  Num examples = 150
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


{'eval_loss': 4.715660572052002,
 'eval_model_preparation_time': 0.0046,
 'eval_runtime': 2.4718,
 'eval_samples_per_second': 60.684,
 'eval_steps_per_second': 3.236}

In [21]:
trainer.train()

***** Running training *****
  Num examples = 850
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 4,250
  Number of trainable parameters = 124,439,808
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,0.432000,0.445846,0.004600
2,0.453300,0.435233,0.004600
3,0.423600,0.434290,0.004600
4,0.393000,0.424289,0.004600
5,0.388300,0.427636,0.004600



***** Running Evaluation *****
  Num examples = 150
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Saving model checkpoint to ./baseline_till_assistant_model_v1/checkpoint-425
Configuration saved in ./baseline_till_assistant_model_v1/checkpoint-425/config.json
Configuration saved in ./baseline_till_assistant_model_v1/checkpoint-425/generation_config.json
Model weights saved in ./baseline_till_assistant_model_v1/checkpoint-425/model.safetensors

***** Running Evaluation *****
  Num examples = 150
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
Saving model checkpoint to ./baseline_till_assistant_model_v1/checkpoint-850
Configuration saved in ./baseline_till_assistant_model_v1/checkpoint-850/config.json
Configuration saved in ./baseline_till_assistant_model_v1/checkpoint-850/generation_config.json
Model weights saved in ./baseline_till_assistant_model_v1/checkpoint-850/model.safetensors



Epoch,Training Loss,Validation Loss,Model Preparation Time
1,0.432000,0.445846,0.004600
2,0.453300,0.435233,0.004600
3,0.423600,0.434290,0.004600
4,0.393000,0.424289,0.004600
5,0.388300,0.427636,0.004600
6,0.363300,0.427032,0.004600
7,0.396000,0.430162,0.004600
8,0.435500,0.435046,0.004600
9,0.329200,0.441049,0.004600
10,0.380400,0.445306,0.004600



***** Running Evaluation *****
  Num examples = 150
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
Saving model checkpoint to ./baseline_till_assistant_model_v1/checkpoint-2550
Configuration saved in ./baseline_till_assistant_model_v1/checkpoint-2550/config.json
Configuration saved in ./baseline_till_assistant_model_v1/checkpoint-2550/generation_config.json
Model weights saved in ./baseline_till_assistant_model_v1/checkpoint-2550/model.safetensors

***** Running Evaluation *****
  Num examples = 150
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
Saving model checkpoint to ./baseline_till_as

TrainOutput(global_step=4250, training_loss=0.4306421164344339, metrics={'train_runtime': 458.3331, 'train_samples_per_second': 18.545, 'train_steps_per_second': 9.273, 'total_flos': 162958450176000.0, 'train_loss': 0.4306421164344339, 'epoch': 10.0})

In [22]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 150
  Batch size = 20
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


{'eval_loss': 0.42428892850875854,
 'eval_model_preparation_time': 0.0046,
 'eval_runtime': 0.6645,
 'eval_samples_per_second': 225.745,
 'eval_steps_per_second': 12.04,
 'epoch': 10.0}

In [ ]:
trainer.save_model()

In [23]:
loaded_model = GPT2LMHeadModel.from_pretrained('././baseline_till_assistant_model_v1/checkpoint-1700') # change the directory to "./baseline_till_assistant_model_v1"
order_generator = pipeline('text-generation', model=BKUK_TILL, tokenizer=tokenizer)

loading configuration file ././baseline_till_assistant_model_v1/checkpoint-1700/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "float32",
  "transformers_version": "4.44.2",
  "use_cache": true,
 

In [27]:
text_sample = 'I want to order chilli bites, nuggets, royale and some fries'
conversion_text_sample = f'{CONVERSION_PROMPT}Query: {text_sample}\n{CONVERSION_TOKEN_1}'

print(conversion_text_sample)

TILL
Query: I want to order chilli bites, nuggets, royale and some fries
Order


In [31]:
print(order_generator(
    conversion_text_sample, num_beams=50, early_stopping=True, temperature=0.5,
    max_length=len(tokenizer.encode(conversion_text_sample)) + 20
)[0]['generated_text'])

TILL
Query: I want to order chilli bites, nuggets, royale and some fries
Order: Sure! You've ordered Chilli Bites, Nuggets, Royale. You've ordered Chilli


In [ ]:
# Now using prompt engineering, I will create a model that will handle multiple tasks at once
# The model will give till response and also record the orders customer made

In [32]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, GPT2LMHeadModel, pipeline, \
                         GPT2Tokenizer
import pandas as pd
from datasets import Dataset

In [45]:
MODEL = 'distilgpt2'

tokenizer = GPT2Tokenizer.from_pretrained(MODEL)

tokenizer.pad_token = tokenizer.eos_token

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/2290a62682d06624634c1f46a6ad5be0f47f38aa/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/2290a62682d06624634c1f46a6ad5be0f47f38aa/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/2290a62682d06624634c1f46a6ad5be0f47f38aa/tokenizer_config.json
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/2290a62682d06624634c1f46a6ad5be0f47f38aa/tokenizer.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/2290a62682d06624634c1f46a6ad5be0f47f38aa/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gel

In [107]:
#add two prompts, one for each task
QUERY_PROMPT = 'Query Task'
RECORD_PROMPT = 'Record Task'
QUERY_TOKEN = 'Response'
RECORD_TOKEN = 'Record'

# This is our "training prompt" that we want GPT2 to recognize and learn
query_task = f'{QUERY_PROMPT}\nQuery: ' + customer_dataset['Customer_Speaking'] + '\n' + QUERY_TOKEN + ': ' + customer_dataset['Till_Response'].astype(str)
record_task = f'{RECORD_PROMPT}\nQuery: ' + customer_dataset['Customer_Speaking'] + '\n' + RECORD_TOKEN + ': ' + customer_dataset['Ordered_Items'].astype(str)
print(query_task[0])
print(f"\n{record_task[0]}")

Query Task
Query: I need Fries, 6 Nuggets, 3 Chilli Bites
Response: Sure! You've ordered Fries, 6 Nuggets, 3 Chilli Bites.

Record Task
Query: I need Fries, 6 Nuggets, 3 Chilli Bites
Record: 3 chilli bites, 6 nuggets, fries


In [108]:
task_df = pd.DataFrame({"query_text": query_task,
                        "record_text": record_task})
task_df.head()

,query_text,record_text
0,"Query Task\nQuery: I need Fries, 6 Nuggets, 3 ...","Record Task\nQuery: I need Fries, 6 Nuggets, 3..."
1,"Query Task\nQuery: I need Bacon Stacker, mediu...","Record Task\nQuery: I need Bacon Stacker, medi..."
2,Query Task\nQuery: I need Large Fries\nRespons...,Record Task\nQuery: I need Large Fries\nRecord...
3,Query Task\nQuery: Please give me small Hot Ch...,Record Task\nQuery: Please give me small Hot C...
4,"Query Task\nQuery: I'll have Diet Cocacola, St...","Record Task\nQuery: I'll have Diet Cocacola, S..."


In [113]:
training_examples = task_df['query_text'].tolist() + task_df['record_text'].tolist()
print(len(training_examples))

AttributeError: 'function' object has no attribute 'append'

In [111]:
print(training_examples[-1])

Record Task
Query: Please give me Whopper, 6 Chilli Bites, 4 Nuggets
Record: whopper, 6 chilli bites, 4 nuggets


In [90]:
multi_task_df = pd.DataFrame({'text': training_examples})

data = Dataset.from_pandas(multi_task_df)

def preprocess(examples):
    return tokenizer(examples['text'], truncation=True)

data = data.map(preprocess, batched=True)

data = data.train_test_split(train_size=.8)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [91]:
model = GPT2LMHeadModel.from_pretrained(MODEL)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilgpt2/snapshots/2290a62682d06624634c1f46a6ad5be0f47f38aa/config.json
Model config GPT2Config {
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
   

In [92]:
training_args = TrainingArguments(
    output_dir="./gpt2_multitask", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=32,  # batch size for evaluation
    logging_steps=50,
    load_best_model_at_end=True,
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test'],
    data_collator=data_collator
)

trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).

***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


{'eval_loss': 5.733813285827637,
 'eval_model_preparation_time': 0.0023,
 'eval_runtime': 1.1423,
 'eval_samples_per_second': 350.178,
 'eval_steps_per_second': 11.381}

In [93]:
trainer.train()

***** Running training *****
  Num examples = 1,600
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 81,912,576
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Model Preparation Time
1,1.898200,0.602493,0.002300
2,0.623600,0.500620,0.002300
3,0.560100,0.486375,0.002300
4,0.536000,0.468972,0.002300
5,0.510500,0.465466,0.002300
6,0.496800,0.467134,0.002300
7,0.487200,0.461381,0.002300
8,0.481900,0.458361,0.002300
9,0.475500,0.458161,0.002300
10,0.473400,0.455632,0.002300



***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


Saving model checkpoint to ./gpt2_multitask/checkpoint-50
Configuration saved in ./gpt2_multitask/checkpoint-50/config.json
Configuration saved in ./gpt2_multitask/checkpoint-50/generation_config.json
Model weights saved in ./gpt2_multitask/checkpoint-50/model.safetensors

***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
Saving model checkpoint to ./gpt2_multitask/checkpoint-100
Configuration saved in ./gpt2_multitask/checkpoint-100/config.json
Configuration saved in ./gpt2_multitask/checkpoint-100/generation_config.json
Model weights saved in ./gpt2_multitask/checkpoint-100/model.safetensors

***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argumen

TrainOutput(global_step=500, training_loss=0.6543276062011719, metrics={'train_runtime': 323.5458, 'train_samples_per_second': 49.452, 'train_steps_per_second': 1.545, 'total_flos': 192837001936896.0, 'train_loss': 0.6543276062011719, 'epoch': 10.0})

In [94]:
trainer.evaluate()


***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: text. If text are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.


{'eval_loss': 0.45563194155693054,
 'eval_model_preparation_time': 0.0023,
 'eval_runtime': 1.0892,
 'eval_samples_per_second': 367.235,
 'eval_steps_per_second': 11.935,
 'epoch': 10.0}

In [95]:
trainer.save_model()

Saving model checkpoint to ./gpt2_multitask
Configuration saved in ./gpt2_multitask/config.json
Configuration saved in ./gpt2_multitask/generation_config.json
Model weights saved in ./gpt2_multitask/model.safetensors


In [96]:
loaded_model = GPT2LMHeadModel.from_pretrained('./gpt2_multitask')

generator = pipeline('text-generation', model=loaded_model, tokenizer=tokenizer)

loading configuration file ./gpt2_multitask/config.json
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype": "f

In [71]:
query_sample, response, order_list = customer_dataset.sample(1)[['Customer_Speaking', 'Till_Response', 'Ordered_Items']].values[0]

print(query_sample)
print(response)
print(order_list)

num_tokens = len(tokenizer(text_sample)['input_ids'])
print(num_tokens)

Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce
Sure! You've ordered 9 Doritos, Vegan Royale, Barbeque Sauce.
vegan royale, 9 doritos, barbeque sauce
16


In [137]:
query_text_sample = f'{QUERY_PROMPT}\nQuery: {query_sample}\n'
order_text_sample = f'{RECORD_PROMPT}\nQuery: {text_sample}\n'
print(query_text_sample)
print(order_text_sample)

Query Task
Query: Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce

Record Task
Query: I want to order chilli bites, nuggets, royale and some fries



In [138]:
for generated_text in generator(query_text_sample, num_return_sequences=5, max_length=num_tokens + 20):
    print(generated_text['generated_text'])
    print("-------------")

Query Task
Query: Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce
Response:: Sure! You've ordered 9 Doritos, Vegan
-------------
Query Task
Query: Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce
Response:: Sure! You've ordered 9 Doritos, Vegan
-------------
Query Task
Query: Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce
Response:: Sure! You've ordered 9 Doritos, Vegan
-------------
Query Task
Query: Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce
Response:: Sure! You've ordered 9 Doritos, Vegan
-------------
Query Task
Query: Can you get me 9 Doritos, Vegan Royale, medium Barbeque Sauce
Response:: Sure! You've ordered 9 Doritos, Vegan
-------------


In [139]:
for generated_text in generator(order_text_sample, num_return_sequences=5, max_length=num_tokens + 20):
    print(generated_text['generated_text'])
    print("---------------")

Record Task
Query: I want to order chilli bites, nuggets, royale and some fries
Record:: royale, nuggets, fries, chilli bites,
---------------
Record Task
Query: I want to order chilli bites, nuggets, royale and some fries
Record:: royale, nuggets, fries, chilli bites,
---------------
Record Task
Query: I want to order chilli bites, nuggets, royale and some fries
Record:: royale, chilli bites, fries, mayo,
---------------
Record Task
Query: I want to order chilli bites, nuggets, royale and some fries
Record:: royale, mayo, fries, large fries, n
---------------
Record Task
Query: I want to order chilli bites, nuggets, royale and some fries
Record:: royale, nuggets, fries, tea, fries,
---------------
